## 准备数据

In [33]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [34]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [43]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(initial_value=tf.random.normal(shape=[28*28,100],dtype=tf.float32))
        self.W2 = tf.Variable(initial_value=tf.random.normal(shape=[100,10],dtype=tf.float32))
        self.b1 = tf.Variable(initial_value=tf.zeros(shape=[100],dtype=tf.float32))
        self.b2 = tf.Variable(initial_value=tf.zeros(shape=[10],dtype=tf.float32))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x_new = tf.reshape(x,[-1,28*28])#展平成二维
        hidden = tf.nn.relu(tf.matmul(x_new,self.W1)+self.b1)
        logits = tf.matmul(hidden,self.W2)+self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [41]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.05*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [44]:
train_data, test_data = mnist_dataset()


for epoch in range(200):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 105.43903 ; accuracy 0.0931
epoch 1 : loss 70.10627 ; accuracy 0.10208333
epoch 2 : loss 59.168552 ; accuracy 0.124066666
epoch 3 : loss 52.162067 ; accuracy 0.14836666
epoch 4 : loss 46.495224 ; accuracy 0.16938333
epoch 5 : loss 41.667667 ; accuracy 0.19071667
epoch 6 : loss 37.639324 ; accuracy 0.2117
epoch 7 : loss 34.40866 ; accuracy 0.23398334
epoch 8 : loss 31.806759 ; accuracy 0.25678334
epoch 9 : loss 29.608688 ; accuracy 0.27785
epoch 10 : loss 27.673944 ; accuracy 0.29715
epoch 11 : loss 25.955088 ; accuracy 0.31753334
epoch 12 : loss 24.43153 ; accuracy 0.33601665
epoch 13 : loss 23.075996 ; accuracy 0.35325
epoch 14 : loss 21.867767 ; accuracy 0.37115
epoch 15 : loss 20.79315 ; accuracy 0.38788334
epoch 16 : loss 19.83434 ; accuracy 0.40313333
epoch 17 : loss 18.97701 ; accuracy 0.4181
epoch 18 : loss 18.207842 ; accuracy 0.43218333
epoch 19 : loss 17.513494 ; accuracy 0.44516668
epoch 20 : loss 16.885286 ; accuracy 0.45756668
epoch 21 : loss 16.313917 ; acc